In [36]:
# Add path to other HW folders
# Incites modular design / No need to copy ReadFile into Homework3 folder
import sys
sys.path.append("../")

# Import Functions
from Homework2.ReadFile import Read
import numpy as np
import pandas as pd

In [28]:
# You may be wondering why I went through so much trouble in making GetParticleType:
# I like being able to use strings. My background is FITS image processing, so I'm used to dealing
# with strings because of the header files... Yes I will be reusing this.

def GetParticleType(ptype):
    """
    Function to parse particle type parameter.
    `ptype` can be an integer (1-3) or string (halo, disk, bulge).

    :param ptype: particle type => halo (1), disk (2), or bulge (3) 
    :return: parsed particle type
    """
    # If ptype is string, then reassign according to indexing rule
    if isinstance(ptype, str):
        if ptype.lower()=='halo':
            ptype = 1
        elif ptype.lower()=='disk':
            ptype = 2
        elif ptype.lower()=='bulge':
            ptype = 3
        else:
            raise ValueError('Wrong particle type')
    
    # else assume it is an integer; check it's within range
    else:
        if not (1 <= ptype <= 3):
            raise ValueError('Particle type out of range (1,3)')
    return ptype


def GalaxyMass(filename, ptype):
    """
    Read in MK data file, and return the total mass of any 
    desired galaxy component: Halo (1), Disk (2), Bulge (3)
    
    `ptype` can be integer or string.

    :param filename: file path to data file
    :param ptye: particle type; halo (1), disk (2), or bulge (3) 
    :return: Total component mass in 10^12 Msun
    
    """
    # Read data in
    time, total, data = Read(filename)
    
    # Mask of component mass using parsed ptype
    mask = data['type'] == GetParticleType(ptype)

    # Now get the mass of all particles in component in 10^10 Msun
    masses = data[mask]['m']

    # Sum up masses; Divide by 100 to express in 10^12 Msun.
    total_mass = masses.sum() / 100

    # Return rounded mass value
    return np.round(total_mass, 3)

In [67]:
# These are the 'testing' locations of the galaxies data
mw_path = '../../SampleData/MW_000.txt'
m31_path = '../../SampleData/M31_000.txt'
m33_path = '../../SampleData/M33_000.txt'

In [65]:
# List of Galaxy names
names = 'MilkyWay', 'M31', 'M33'

# List of total mass components for the Milky Way: Halo, Disk, and Bulge
mw_masses = ( GalaxyMass(mw_path, ptype) for ptype in [1, 2, 3] )
m31_masses = ( GalaxyMass(m31_path, ptype) for ptype in [1, 2, 3] )
m33_masses = ( GalaxyMass(m33_path, ptype) for ptype in [1, 2] )

# Column labels! According to assignment
columns = ['Galaxy Name', 'Halo Mass (10^12 Msun)', 'Disk Mass (10^12 Msun)', 
           'Bulge Mass (10^12 Msun)', 'Total (10^12 Msun)', 'f_bar']

# Pass data to a Pandas DataFrame, create initial table
mass_breakdown = pd.DataFrame([['MilkyWay', *mw_masses, 0, 0], 
                               ['M31', *m31_masses, 0, 0], 
                               ['M33', *m33_masses, 0, 0] ], 
                              columns=columns)
mass_breakdown

,Galaxy Name,Halo Mass (10^12 Msun),Disk Mass (10^12 Msun),Bulge Mass (10^12 Msun),Total (10^12 Msun),f_bar
0,MilkyWay,1.975,0.075,0.010,0,0.0
1,M31,1.921,0.120,0.019,0,0.0
2,M33,0.187,0.009,0.000,0,NaN


In [66]:
# Index component mass columns (1,2,3), and sum rowwise to get total mass per galaxy (10^12 Msun)
total_masses = mass_breakdown[columns[1:4]].sum(axis=1)

# Reassign values for total mass column
mass_breakdown['Total (10^12 Msun)'] = total_masses
mass_breakdown

,Galaxy Name,Halo Mass (10^12 Msun),Disk Mass (10^12 Msun),Bulge Mass (10^12 Msun),Total (10^12 Msun),f_bar
0,MilkyWay,1.975,0.075,0.010,2.060,0.0
1,M31,1.921,0.120,0.019,2.060,0.0
2,M33,0.187,0.009,0.000,0.196,NaN


In [73]:
# Baryon mass per galaxy
baryon_masses = mass_breakdown[columns[2:4]].sum(axis=1)

# baryon fraction per galaxy
f_bar = baryon_masses / total_masses

# Local group baryon mass, and DM mass
local_baryon_mass = baryon_masses.sum()
loca_DM_mass = mass_breakdown['Halo Mass (10^12 Msun)'].sum()

# Total local group masss (10^12 Msun)
local_mass = total_masses.sum()

# local group baryon fraction
local_fbar = local_baryon_mass / local_mass

# Reassign values for baryon fraction mass
mass_breakdown['f_bar'] = f_bar

print(f'Local Group Baryon Mass = \t{local_fbar}')
mass_breakdown

Local Group Baryon Mass = 	0.053985171455050975


,Galaxy Name,Halo Mass (10^12 Msun),Disk Mass (10^12 Msun),Bulge Mass (10^12 Msun),Total (10^12 Msun),f_bar
0,MilkyWay,1.975,0.075,0.010,2.060,0.041262
1,M31,1.921,0.120,0.019,2.060,0.067476
2,M33,0.187,0.009,0.000,0.196,0.045918


In [77]:
print(f"MW to M31 Halo Mass Ratio = \t{mass_breakdown['Halo Mass (10^12 Msun)'][0]/mass_breakdown['Halo Mass (10^12 Msun)'][1]}")
print(f'MW to M31 Baryon Mass Ratio = \t{baryon_masses[0]/baryon_masses[1]}')

MW to M31 Halo Mass Ratio = 	1.028110359187923
MW to M31 Baryon Mass Ratio = 	0.6115107913669064


## Questions (some of the questions sort of overlap)

1. How does the total mass of the MW and M31 compare in this simulation? What galaxy component dominates this total mass?

Their total mass are equal! However M31's baryon mass is slighly higher than the MW, and the opposite for the halo mass. This baryon mass is mostly concentrated on the buldge; M31's bulge is ~x2 heavier than MW's bulge.

2. How does the stellar mass of the MW and M31 compare? Which galaxy do you expect to be more luminous?

Andromeda is expected to be brighter due to the slighly higher baryon mass ratio.

3. How does the total dark matter mass of MW and M31 compare in this simulation (ratio)? Is this surprising, given their difference in stellar mass?

The MW to M31 Halo mass ratio is ~1.03. While the baryon ratio is ~0.61. This means that the MW's dark matter mass makes up the lack of baryon. However, it is suprising that the ratios are so different. They essentially have the same dark matter mass; so they should probably also have the same baryon mass.

4. What is the ratio of stellar mass to total mass for each galaxy (F_bar)? In the Universe, $\Omega_b/\Omega_m$ ~ 16%. of all mass is locked up in baryons (gas & stars) vs dark matter. How does this ratio compare to the baryon fraction you computer for each galaxy? Given than the total gas mass in the disk of these galaxies is negligible compared to the stellar mass, any ideas for why he universal baryon fraction might differ from that in these galaxies?

The baryon fractions are on the table above. Our $f_{bar}$ values are ~1/2 and 1/3 of the universal baryon fraction. Throughout the galaxy's life, the baryon mass fraction changes due to the mainly gravitational interaction between baryon and dark matter; baryon particles interact in many more ways than dark matter. This interaction can take in form of AGN feedback & outflows.

In [80]:
mass_breakdown.to_pd